# Biomass Mapping

In [1]:
import pandas as pd
import geopandas as gpd

Now I will import the csv file containing data for each crop for Agro-Climatic Potential Yield


In [3]:
df= pd.read_csv("./dataset/crop.csv")

In [5]:
print(df.head())

                             Source.Name          Name  \
0  Agro-climatic Potential Yield (1).csv  alfa200a_yld   
1  Agro-climatic Potential Yield (1).csv  alfa200a_yld   
2  Agro-climatic Potential Yield (1).csv  alfa200a_yld   
3  Agro-climatic Potential Yield (1).csv  alfa200a_yld   
4  Agro-climatic Potential Yield (1).csv  alfa200a_yld   

                                      Path to File       Sub-Theme Name  \
0  \res02\NorESM1-M\rcp8p5\2080sH\alfa200a_yld.tif  Agro-climatic Yield   
1  \res02\NorESM1-M\rcp8p5\2050sH\alfa200a_yld.tif  Agro-climatic Yield   
2  \res02\NorESM1-M\rcp8p5\2020sH\alfa200a_yld.tif  Agro-climatic Yield   
3  \res02\NorESM1-M\rcp6p0\2080sH\alfa200a_yld.tif  Agro-climatic Yield   
4  \res02\NorESM1-M\rcp6p0\2050sH\alfa200a_yld.tif  Agro-climatic Yield   

                   Variable Name  \
0  Agro-climatic potential yield   
1  Agro-climatic potential yield   
2  Agro-climatic potential yield   
3  Agro-climatic potential yield   
4  Agro-climatic pot